## Imports

In [1]:
import numpy as np # for stats
import random # for randomly generating target and start words
# import operator # for sorting letter frequency distribution
# import time # for #dramaticeffect
import pandas as pd
from nltk.corpus import movie_reviews, treebank, brown, gutenberg, switchboard
from wordle_functions import *
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import make_pipeline

## Importing datasets

### official words
- official wordle word list

In [2]:
### Official list
official_words = []

with open("data/official_words_processed.txt", "r", encoding = "utf-8") as f:
    for word in f.read().split("\n"):
        official_words.append(word)

f.close() # closes connection to file

print(len(official_words))
official_words[:5]

2310


['wince', 'thyme', 'mower', 'horde', 'heard']

In [6]:
### Official list
official_words = []

with open("data/official_words_processed.txt", "r", encoding = "utf-8") as f:
    for word in f.read().split("\n"):
        if len(word) > 0: # there's one blank entry at the start
            official_words.append(word)

f.close() # closes connection to file

print(len(official_words))
official_words[:10]

2309


['wince',
 'thyme',
 'mower',
 'horde',
 'heard',
 'tenor',
 'zonal',
 'parry',
 'shied',
 'fizzy']

## Creating or Importing Training Data

In [7]:
### Generating list of random words

random_words_1 = []

num_words = 20

for i in range(0, num_words):
    r_int = random.randint(0, len(official_words) - 1)
    guess = official_words[r_int]
    random_words_1.append(guess)

print(len(random_words_1))
print(random_words_1[:5])

20
['gnome', 'wrote', 'wrath', 'waist', 'sieve']


In [8]:
# pd.read_csv("data/sims_df_completed.csv").drop(columns = "Unnamed: 0")

In [9]:
try:
    sims_df = pd.read_csv("data/sims_df_complete.csv").drop(columns = "Unnamed: 0")
except:
    FileNotFoundError
        

    # takes 1min38s for all words x one word

    ## Generating lists of random words
    random_words = []

    num_words = 1000

    for i in range(0, num_words):
        r_int = random.randint(0, len(official_words) - 1)
        guess = official_words[r_int]
        random_words.append(guess)

    excepts = []

    stats_master = {}

    for start_word in random_words:

        try:
            complete = wordle_wizard(word_list = official_words, max_guesses = 15, 
                guess = start_word, target = "glide", bias = 'entropy',
                random_guess = False, random_target = True, 
                    verbose = False, drama = 0, return_stats = True)
            
        except:
            IndexError
            excepts.append((complete["first_guess"], complete["target_word"]))
            
        for metric, result in complete.items():
            if metric in stats_master.keys():
                stats_master[metric].append(result)
            else:
                stats_master[metric] = []
            
    sims_df = pd.DataFrame(stats_master)
    sims_df.to_csv('data/sims_df_complete.csv')
    print(f"{len(sims_df)} iterations run. {len(excepts)} combinations excepted.")

print(sims_df.shape)
sims_df.head()

(2308, 20)


,first_guess,target_word,first_guess_vowels,first_guess_consonants,target_vowels,target_consonants,target_entropy,first_guess_entropy,target_guessed,mid_guesses_avg_vows,mid_guesses_avg_cons,avg_perf_letters,avg_wrong_pos_letters,avg_wrong_letters,avg_remaining,avg_intermediate_guess_entropy,luck,bias,valid_success,num_guesses
0,detox,glide,2.0,3.0,2.0,3.0,63.75,58.53,True,2.25,2.75,4.0,12.0,14.0,27.00,89.63,0.69,entropy,True,4.0
1,whiff,glide,1.0,4.0,2.0,3.0,63.75,9.47,True,2.40,2.60,8.0,2.0,27.0,51.25,81.89,0.65,entropy,True,5.0
2,soggy,glide,2.0,3.0,2.0,3.0,63.75,29.40,True,2.25,2.75,4.0,9.0,16.0,27.33,82.35,0.71,entropy,True,4.0
3,front,glide,1.0,4.0,2.0,3.0,63.75,59.66,True,2.00,3.00,1.0,2.0,12.0,227.50,86.55,0.78,entropy,True,3.0
4,stein,glide,2.0,3.0,2.0,3.0,63.75,80.00,True,2.33,2.67,0.0,7.0,8.0,41.00,93.33,0.76,entropy,True,3.0


In [10]:
compared_df = pd.read_csv("compared_data/players_compared.csv").drop(columns = "Unnamed: 0")
compared_df

,first_guess,target_word,player,num_guesses,expected_guesses,luck,first_guess_vowels,first_guess_consonants,target_vowels,target_consonants,...,target_entropy,target_guessed,mid_guesses_avg_vows,mid_guesses_avg_cons,avg_perf_letters,avg_wrong_pos_letters,avg_wrong_letters,avg_remaining,avg_intermediate_guess_entropy,valid_success
0,arose,vague,diane,5.0,5.0,0.00,3.0,2.0,3.0,2.0,...,58.25,True,2.60,2.40,6.0,9.0,26.0,18.75,99.86,True
1,arose,vague,wizard,5.0,5.0,0.00,3.0,2.0,3.0,2.0,...,58.25,True,2.60,2.40,7.0,4.0,30.0,19.00,99.86,True
2,douce,apple,aidan,5.0,5.0,0.07,3.0,2.0,2.0,3.0,...,62.83,True,2.60,2.40,5.0,11.0,30.0,49.50,92.75,True
3,douce,apple,wizard,5.0,5.0,0.00,3.0,2.0,2.0,3.0,...,62.83,True,2.40,2.60,8.0,7.0,28.0,48.00,92.75,True
4,douce,ninth,aidan,4.0,5.0,0.27,3.0,2.0,1.0,4.0,...,35.38,True,2.00,3.00,1.0,2.0,27.0,123.00,90.92,True
5,douce,ninth,wizard,5.0,5.0,0.00,3.0,2.0,1.0,4.0,...,35.38,True,1.80,3.20,3.0,12.0,35.0,95.00,92.74,True
6,douce,flail,aidan,4.0,4.0,0.09,3.0,2.0,2.0,3.0,...,42.30,True,2.50,2.50,3.0,6.0,21.0,122.33,90.92,True
7,douce,flail,wizard,4.0,4.0,0.00,3.0,2.0,2.0,3.0,...,42.30,True,2.25,2.75,6.0,0.0,21.0,120.00,90.92,True
8,douce,stage,aidan,4.0,4.0,0.09,3.0,2.0,2.0,3.0,...,81.29,True,2.50,2.50,4.0,3.0,22.0,65.67,90.92,True
9,douce,stage,wizard,4.0,4.0,0.00,3.0,2.0,2.0,3.0,...,81.29,True,2.50,2.50,7.0,2.0,17.0,59.33,90.92,True


In [11]:
players = compared_df['player'].unique().tolist()
players

['diane', 'wizard', 'aidan', 'dad', 'kai']

In [12]:
diane_df = compared_df.query("player == 'diane'")
diane_df

,first_guess,target_word,player,num_guesses,expected_guesses,luck,first_guess_vowels,first_guess_consonants,target_vowels,target_consonants,...,target_entropy,target_guessed,mid_guesses_avg_vows,mid_guesses_avg_cons,avg_perf_letters,avg_wrong_pos_letters,avg_wrong_letters,avg_remaining,avg_intermediate_guess_entropy,valid_success
0,arose,vague,diane,5.0,5.0,0.00,3.0,2.0,3.0,2.0,...,58.25,True,2.60,2.40,6.0,9.0,26.0,18.75,99.86,True
42,audio,worse,diane,4.0,4.0,0.09,4.0,1.0,2.0,3.0,...,76.40,True,2.50,2.50,4.0,8.0,17.0,96.67,90.50,True
43,audio,polka,diane,3.0,3.0,0.13,4.0,1.0,2.0,3.0,...,54.91,True,2.67,2.33,3.0,5.0,7.0,38.50,87.34,True
49,audio,moose,diane,4.0,6.0,0.40,4.0,1.0,3.0,2.0,...,53.53,True,2.75,2.25,5.0,5.0,18.0,98.00,90.51,True
52,audio,above,diane,3.0,4.0,0.34,4.0,1.0,3.0,2.0,...,65.89,True,3.33,1.67,4.0,2.0,8.0,16.50,87.35,True
54,audio,squat,diane,3.0,3.0,0.13,4.0,1.0,2.0,3.0,...,50.65,True,3.00,2.00,2.0,6.0,7.0,16.50,87.35,True


In [13]:
wizard_df = compared_df.query("player == 'wizard'")
wizard_df

,first_guess,target_word,player,num_guesses,expected_guesses,luck,first_guess_vowels,first_guess_consonants,target_vowels,target_consonants,...,target_entropy,target_guessed,mid_guesses_avg_vows,mid_guesses_avg_cons,avg_perf_letters,avg_wrong_pos_letters,avg_wrong_letters,avg_remaining,avg_intermediate_guess_entropy,valid_success
1,arose,vague,wizard,5.0,5.0,0.0,3.0,2.0,3.0,2.0,...,58.25,True,2.60,2.40,7.0,4.0,30.0,19.00,99.86,True
3,douce,apple,wizard,5.0,5.0,0.0,3.0,2.0,2.0,3.0,...,62.83,True,2.40,2.60,8.0,7.0,28.0,48.00,92.75,True
5,douce,ninth,wizard,5.0,5.0,0.0,3.0,2.0,1.0,4.0,...,35.38,True,1.80,3.20,3.0,12.0,35.0,95.00,92.74,True
7,douce,flail,wizard,4.0,4.0,0.0,3.0,2.0,2.0,3.0,...,42.30,True,2.25,2.75,6.0,0.0,21.0,120.00,90.92,True
9,douce,stage,wizard,4.0,4.0,0.0,3.0,2.0,2.0,3.0,...,81.29,True,2.50,2.50,7.0,2.0,17.0,59.33,90.92,True
11,douce,heady,wizard,4.0,4.0,0.0,3.0,2.0,3.0,2.0,...,66.43,True,2.50,2.50,0.0,15.0,15.0,21.67,90.92,True
13,douce,usage,wizard,3.0,3.0,0.0,3.0,2.0,3.0,2.0,...,73.67,True,3.00,2.00,2.0,5.0,7.0,18.00,87.91,True
15,douce,sound,wizard,3.0,3.0,0.0,3.0,2.0,2.0,3.0,...,50.14,True,2.33,2.67,6.0,2.0,5.0,8.50,87.91,True
17,douce,salsa,wizard,4.0,4.0,0.0,3.0,2.0,2.0,3.0,...,35.65,True,2.25,2.75,3.0,4.0,23.0,123.67,90.92,True
19,douce,magic,wizard,4.0,4.0,0.0,3.0,2.0,2.0,3.0,...,46.90,True,2.25,2.75,4.0,7.0,18.0,27.00,90.92,True


In [14]:
aidan_df = compared_df.query("player == 'aidan'")
aidan_df

,first_guess,target_word,player,num_guesses,expected_guesses,luck,first_guess_vowels,first_guess_consonants,target_vowels,target_consonants,...,target_entropy,target_guessed,mid_guesses_avg_vows,mid_guesses_avg_cons,avg_perf_letters,avg_wrong_pos_letters,avg_wrong_letters,avg_remaining,avg_intermediate_guess_entropy,valid_success
2,douce,apple,aidan,5.0,5.0,0.07,3.0,2.0,2.0,3.0,...,62.83,True,2.60,2.40,5.0,11.0,30.0,49.50,92.75,True
4,douce,ninth,aidan,4.0,5.0,0.27,3.0,2.0,1.0,4.0,...,35.38,True,2.00,3.00,1.0,2.0,27.0,123.00,90.92,True
6,douce,flail,aidan,4.0,4.0,0.09,3.0,2.0,2.0,3.0,...,42.30,True,2.50,2.50,3.0,6.0,21.0,122.33,90.92,True
8,douce,stage,aidan,4.0,4.0,0.09,3.0,2.0,2.0,3.0,...,81.29,True,2.50,2.50,4.0,3.0,22.0,65.67,90.92,True
10,douce,heady,aidan,5.0,4.0,-0.16,3.0,2.0,3.0,2.0,...,66.43,True,3.00,2.00,9.0,11.0,22.0,16.00,92.74,True
12,douce,usage,aidan,5.0,3.0,-0.55,3.0,2.0,3.0,2.0,...,73.67,True,2.40,2.60,5.0,14.0,31.0,10.00,92.75,True
14,douce,sound,aidan,6.0,3.0,-0.88,3.0,2.0,2.0,3.0,...,50.14,True,2.17,2.83,18.0,5.0,20.0,6.40,93.96,True
16,douce,salsa,aidan,6.0,4.0,-0.41,3.0,2.0,2.0,3.0,...,35.65,True,1.83,3.17,10.0,7.0,47.0,74.20,93.95,True
18,douce,magic,aidan,5.0,4.0,-0.16,3.0,2.0,2.0,3.0,...,46.90,True,2.40,2.60,7.0,9.0,29.0,21.75,92.73,True
20,douce,avail,aidan,5.0,5.0,0.07,3.0,2.0,3.0,2.0,...,40.08,True,2.60,2.40,6.0,10.0,31.0,92.50,92.73,True


In [15]:
dad_df = compared_df.query("player == 'dad'")
dad_df

,first_guess,target_word,player,num_guesses,expected_guesses,luck,first_guess_vowels,first_guess_consonants,target_vowels,target_consonants,...,target_entropy,target_guessed,mid_guesses_avg_vows,mid_guesses_avg_cons,avg_perf_letters,avg_wrong_pos_letters,avg_wrong_letters,avg_remaining,avg_intermediate_guess_entropy,valid_success
38,audio,syrup,dad,4.0,3.0,-0.21,4.0,1.0,2.0,3.0,...,49.32,True,2.50,2.50,1.0,10.0,18.0,37.00,90.50,True
40,audio,worse,dad,4.0,4.0,0.09,4.0,1.0,2.0,3.0,...,76.40,True,2.50,2.50,5.0,9.0,15.0,95.33,90.50,True
45,audio,polka,dad,4.0,3.0,-0.21,4.0,1.0,2.0,3.0,...,55.00,True,2.50,2.50,2.0,11.0,17.0,32.00,90.51,True
47,audio,moose,dad,6.0,6.0,0.06,4.0,1.0,3.0,2.0,...,53.53,True,2.83,2.17,13.0,9.0,37.0,59.80,93.67,True
56,audio,squat,dad,3.0,3.0,0.13,4.0,1.0,2.0,3.0,...,50.67,True,3.00,2.00,1.0,6.0,8.0,17.00,87.34,True


In [16]:
kai_df = compared_df.query("player == 'kai'")
kai_df

,first_guess,target_word,player,num_guesses,expected_guesses,luck,first_guess_vowels,first_guess_consonants,target_vowels,target_consonants,...,target_entropy,target_guessed,mid_guesses_avg_vows,mid_guesses_avg_cons,avg_perf_letters,avg_wrong_pos_letters,avg_wrong_letters,avg_remaining,avg_intermediate_guess_entropy,valid_success
50,homes,squat,kai,3.0,4.0,0.34,2.0,3.0,2.0,3.0,...,50.65,True,1.67,3.33,2.0,4.0,9.0,81.5,88.35,True


## ML

In [17]:
compared_df

,first_guess,target_word,player,num_guesses,expected_guesses,luck,first_guess_vowels,first_guess_consonants,target_vowels,target_consonants,...,target_entropy,target_guessed,mid_guesses_avg_vows,mid_guesses_avg_cons,avg_perf_letters,avg_wrong_pos_letters,avg_wrong_letters,avg_remaining,avg_intermediate_guess_entropy,valid_success
0,arose,vague,diane,5.0,5.0,0.00,3.0,2.0,3.0,2.0,...,58.25,True,2.60,2.40,6.0,9.0,26.0,18.75,99.86,True
1,arose,vague,wizard,5.0,5.0,0.00,3.0,2.0,3.0,2.0,...,58.25,True,2.60,2.40,7.0,4.0,30.0,19.00,99.86,True
2,douce,apple,aidan,5.0,5.0,0.07,3.0,2.0,2.0,3.0,...,62.83,True,2.60,2.40,5.0,11.0,30.0,49.50,92.75,True
3,douce,apple,wizard,5.0,5.0,0.00,3.0,2.0,2.0,3.0,...,62.83,True,2.40,2.60,8.0,7.0,28.0,48.00,92.75,True
4,douce,ninth,aidan,4.0,5.0,0.27,3.0,2.0,1.0,4.0,...,35.38,True,2.00,3.00,1.0,2.0,27.0,123.00,90.92,True
5,douce,ninth,wizard,5.0,5.0,0.00,3.0,2.0,1.0,4.0,...,35.38,True,1.80,3.20,3.0,12.0,35.0,95.00,92.74,True
6,douce,flail,aidan,4.0,4.0,0.09,3.0,2.0,2.0,3.0,...,42.30,True,2.50,2.50,3.0,6.0,21.0,122.33,90.92,True
7,douce,flail,wizard,4.0,4.0,0.00,3.0,2.0,2.0,3.0,...,42.30,True,2.25,2.75,6.0,0.0,21.0,120.00,90.92,True
8,douce,stage,aidan,4.0,4.0,0.09,3.0,2.0,2.0,3.0,...,81.29,True,2.50,2.50,4.0,3.0,22.0,65.67,90.92,True
9,douce,stage,wizard,4.0,4.0,0.00,3.0,2.0,2.0,3.0,...,81.29,True,2.50,2.50,7.0,2.0,17.0,59.33,90.92,True


In [18]:
compared_df.columns.tolist()

['first_guess',
 'target_word',
 'player',
 'num_guesses',
 'expected_guesses',
 'luck',
 'first_guess_vowels',
 'first_guess_consonants',
 'target_vowels',
 'target_consonants',
 'first_guess_entropy',
 'target_entropy',
 'target_guessed',
 'mid_guesses_avg_vows',
 'mid_guesses_avg_cons',
 'avg_perf_letters',
 'avg_wrong_pos_letters',
 'avg_wrong_letters',
 'avg_remaining',
 'avg_intermediate_guess_entropy',
 'valid_success']

### train/test splits

In [19]:
# train and test dfs
train_df, test_df = train_test_split(compared_df, test_size = 0.2, random_state = 100)
print(train_df.shape)
print(test_df.shape)

# train splits
X_train = train_df.drop(columns = ['num_guesses'])
print(X_train.shape)
y_train = train_df['num_guesses']
print(y_train.shape)

# test splits
X_test = test_df.drop(columns = ['num_guesses'])
print(X_test.shape)
y_test = test_df['num_guesses']
print(y_test.shape)

(46, 21)
(12, 21)
(46, 20)
(46,)
(12, 20)
(12,)


In [20]:
print(X_train.columns.tolist())
X_train.head()

['first_guess', 'target_word', 'player', 'expected_guesses', 'luck', 'first_guess_vowels', 'first_guess_consonants', 'target_vowels', 'target_consonants', 'first_guess_entropy', 'target_entropy', 'target_guessed', 'mid_guesses_avg_vows', 'mid_guesses_avg_cons', 'avg_perf_letters', 'avg_wrong_pos_letters', 'avg_wrong_letters', 'avg_remaining', 'avg_intermediate_guess_entropy', 'valid_success']


,first_guess,target_word,player,expected_guesses,luck,first_guess_vowels,first_guess_consonants,target_vowels,target_consonants,first_guess_entropy,target_entropy,target_guessed,mid_guesses_avg_vows,mid_guesses_avg_cons,avg_perf_letters,avg_wrong_pos_letters,avg_wrong_letters,avg_remaining,avg_intermediate_guess_entropy,valid_success
57,audio,squat,wizard,3.0,0.00,4.0,1.0,2.0,3.0,62.03,50.67,True,3.00,2.00,2.0,6.0,7.0,17.00,87.34,True
11,douce,heady,wizard,4.0,0.00,3.0,2.0,3.0,2.0,63.70,66.43,True,2.50,2.50,0.0,15.0,15.0,21.67,90.92,True
5,douce,ninth,wizard,5.0,0.00,3.0,2.0,1.0,4.0,63.70,35.38,True,1.80,3.20,3.0,12.0,35.0,95.00,92.74,True
53,audio,above,wizard,4.0,0.00,4.0,1.0,3.0,2.0,62.04,65.89,True,3.25,1.75,7.0,3.0,15.0,12.00,90.51,True
18,douce,magic,aidan,4.0,-0.16,3.0,2.0,2.0,3.0,63.67,46.90,True,2.40,2.60,7.0,9.0,29.0,21.75,92.73,True


## Column transformations

In [21]:
categorical_features = ['player', 'first_guess', 'target_word']
binary_features = ['target_guessed', 'valid_success']
ordinal_features = ['expected_guesses']

# list of lists -- one ordering (list) for each col
ordinal_feats_order = [
    [1, 2, 3, 4, 5, 6]
    ]

numeric_features = ['expected_guesses', 'luck', 'first_guess_vowels', 'first_guess_consonants',
                    'target_vowels', 'target_consonants', 'first_guess_entropy',
                    'target_entropy', 'mid_guesses_avg_vows', 'mid_guesses_avg_cons',
                    'avg_perf_letters', 'avg_wrong_pos_letters', 'avg_wrong_letters',
                    'avg_remaining', 'avg_intermediate_guess_entropy']

# passthrough_features = ['player'] # not sure if I want to drop these two, or leave them unaffected by preprocessing
drop_features = ['player']

feat_set = set(categorical_features + binary_features 
               + ordinal_features + numeric_features 
               + drop_features) # set so there are no duplicates

# lazy way of checking to make sure I put each feature into a preprocessor list. Dimensionality is low, so not concerned about computational cost
print(set(X_train.columns.tolist()).difference(feat_set))
print(set(feat_set).difference(X_train.columns.tolist()))

preprocessor = make_column_transformer(
    (OneHotEncoder(), categorical_features),
    (OneHotEncoder(drop = 'if_binary'), binary_features),
    (OrdinalEncoder(categories = ordinal_feats_order), ordinal_features),
    (StandardScaler(), numeric_features),
    ("drop", drop_features)
)

X_train_transformed = preprocessor.fit_transform(X_train)

# Encoding y_train to be 0:x instead of string values
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_train

set()
set()


array([1, 2, 3, 2, 3, 1, 2, 2, 1, 2, 0, 2, 2, 2, 1, 2, 1, 2, 3, 1, 3, 2,
       2, 1, 4, 4, 1, 2, 4, 2, 1, 1, 1, 4, 2, 3, 3, 2, 3, 4, 1, 2, 1, 3,
       2, 2])

In [22]:
all_clf = GradientBoostingClassifier(random_state = 100)
all_clf.fit(X_train_transformed, y_train)

GradientBoostingClassifier(random_state=100)

## Scoring Every Permutation of Words

In [ ]:
try:
    sims_df = pd.read_csv("data/sims_df_complete.csv").drop(columns = "Unnamed: 0")
except:
    FileNotFoundError
        

    # takes 1min38s for all words x one word

    ## Generating lists of random words
    random_words = []

    num_words = 1000

    for i in range(0, num_words):
        r_int = random.randint(0, len(official_words) - 1)
        guess = official_words[r_int]
        random_words.append(guess)

    excepts = []

    stats_master = {}

    for start_word in random_words:

        try:
            complete = wordle_wizard(word_list = official_words, max_guesses = 15, 
                guess = start_word, target = "glide", bias = 'entropy',
                random_guess = False, random_target = True, 
                    verbose = False, drama = 0, return_stats = True)
            
        except:
            IndexError
            excepts.append((complete["first_guess"], complete["target_word"]))
            
        for metric, result in complete.items():
            if metric in stats_master.keys():
                stats_master[metric].append(result)
            else:
                stats_master[metric] = []
            
    sims_df = pd.DataFrame(stats_master)
    sims_df.to_csv('data/sims_df_complete.csv')
    print(f"{len(sims_df)} iterations run. {len(excepts)} combinations excepted.")

print(sims_df.shape)
sims_df.head()

(2308, 20)


,first_guess,target_word,first_guess_vowels,first_guess_consonants,target_vowels,target_consonants,target_entropy,first_guess_entropy,target_guessed,mid_guesses_avg_vows,mid_guesses_avg_cons,avg_perf_letters,avg_wrong_pos_letters,avg_wrong_letters,avg_remaining,avg_intermediate_guess_entropy,luck,bias,valid_success,num_guesses
0,detox,glide,2.0,3.0,2.0,3.0,63.75,58.53,True,2.25,2.75,4.0,12.0,14.0,27.00,89.63,0.69,entropy,True,4.0
1,whiff,glide,1.0,4.0,2.0,3.0,63.75,9.47,True,2.40,2.60,8.0,2.0,27.0,51.25,81.89,0.65,entropy,True,5.0
2,soggy,glide,2.0,3.0,2.0,3.0,63.75,29.40,True,2.25,2.75,4.0,9.0,16.0,27.33,82.35,0.71,entropy,True,4.0
3,front,glide,1.0,4.0,2.0,3.0,63.75,59.66,True,2.00,3.00,1.0,2.0,12.0,227.50,86.55,0.78,entropy,True,3.0
4,stein,glide,2.0,3.0,2.0,3.0,63.75,80.00,True,2.33,2.67,0.0,7.0,8.0,41.00,93.33,0.76,entropy,True,3.0
